In [ ]:
import os
import json
"""
for hyp in os.listdir(f"./data/sgdm_wbn/norm_p1_lrfix"):
    p = f"./data/sgdm_wbn/norm_p1_lrfix/{hyp}"
    with open(f"{p}/settings.json","r") as f:
        js = json.load(f)
    
    js["norm"]["norm_fn"] =  "center_norm_uncenter"
    
    new_p = f"./data/sgdm_wbn/cnormu_p1_lrfix/{hyp}"
    os.makedirs(new_p,exist_ok=True)
    with open(f"{new_p}/settings.json","w") as f:
        json.dump(js,f,indent=4)
"""

In [4]:
from jax.tree_util import keystr,tree_map_with_path
def substrings_in_path(s,match):
    """
    Returns True if all strings in *substrings is in s, else False. 

    Takes:
        s <string> : string for which we want to check if it has substrings inside
        *substrings <string> : list of strings for which we want to check if all of the appear in s
    Returns:
        <boolean>
    """
    ors = match.split("|")
    ands = [e_or.split("&") for e_or in ors]

    return any([all([e_and.lower() in keystr(s).lower() for e_and in e_or]) for e_or in ands])

In [7]:
d = {"Dense" : {"out" : [0,1,2]},"Conv" : {"kernel" : [0,4,2],"bias" : [10,4,2]}}
print(tree_map_with_path(lambda s,x : substrings_in_path(s,"conv&kernel|dense&out"),d))

{'Conv': {'bias': [False, False, False], 'kernel': [True, True, True]}, 'Dense': {'out': [True, True, True]}}


In [6]:
js = lambda lr : {
    "num_devices": 1,
    "num_experiments_per_device": 3,
    "random_key": 42,
    "num_steps": 100000,
    "save_args": {
        "save_states_every": 100000,
        "save_train_stats_every": 1000,
        "save_test_stats_every": 1000,
        "save_grad_every": -1,
        "save_hessian_every": -1
    },
    "model": {
        "model": "vgg11",
        "num_classes": 10,
        "activation_fn": "relu",
        "use_bn" : False
    },
    "dataset": {
        "dataset": "cifar10",
        "batch_size": 64,
        "dataset_path": "./data/datasets/"
    },
    "optimizer": {
        "optimizer": "sgdm",
        "lr": lr,
        "lambda_wd": 0,
        "momentum": 0.9,
        "apply_wd_every": 1
    },
    "norm": {
        "change_scale": "identity",
        "norm_fn": "identity",
        "norm_multiply": 1,
        "norm_every": -1,
        "reverse_norms": False
    },
    "at_step" : 0
}

In [7]:
start_lr = 0.0001
for i in range(10):
    lr = round(start_lr*(2**i),8)

    os.makedirs(f"./data/sgdm_wobn/noreg/{str(lr)}",exist_ok=True)
    with open(f"./data/sgdm_wobn/noreg/{str(lr)}/settings.json", "w") as f:
        json.dump(js(lr),f,indent=4)

In [1]:
import pickle as pkl

with open("./test/cnorm_wbn/states/0.pkl","rb") as f:
    s = pkl.load(f)

In [9]:
for optim in ["./data/adam_wbn"]:
    for exp in os.listdir(optim):
        for hyp in os.listdir(optim + "/" + exp):
            if os.path.exists(os.path.join(optim,exp,hyp,"settings.json")):
                with open(os.path.join(optim,exp,hyp,"settings.json"),"r") as f:
                    js = json.load(f)
                
                js["model"]["bn"] = True
                js["dataset"]["dataset_path"] = "./data/datasets/"
                with open(os.path.join(optim,exp,hyp,"settings.json"),"w") as f:
                    json.dump(js,f,indent=4)


In [21]:
names = []
ps = []
trained = []
for hyp in os.listdir("./exps_adam2/wd_div4"):
    with open(os.path.join("./exps_adam2/wd_div4",hyp,"settings.json"),"r") as f:
        js = json.load(f)
    names.append(hyp)
    ps.append(js["optimizer"]["lambda_wd"])
    trained.append(os.path.exists(os.path.join("./exps_adam2/wd_div4",hyp,"test_stats")))

for i in range(len(names)):
    matching = []
    for j in range(len(names)):
        if ps[i] == ps[j] and names[i] != names[j]:
            matching.append(f"[{names[j]} : {ps[j]} | {trained[j]}]")
    print(f"[{names[i]} : {ps[i]} | {trained[i]}] -> {" ".join(matching)}")

[0.002048 : 0.002048 | True] -> 
[3.2e-05 : 3.2e-05 | True] -> 
[2e-06 : 2e-06 | True] -> 
[0.000256 : 0.000256 | True] -> 
[0.032768 : 0.032768 | True] -> 
[4e-06 : 4e-06 | True] -> 
[0.016384 : 0.016384 | True] -> 
[1.6e-05 : 1.6e-05 | True] -> 
[0.001024 : 0.001024 | True] -> 
[0.008192 : 0.008192 | True] -> 
[1e-06 : 1e-06 | True] -> 
[0.065536 : 0.065536 | True] -> 
[0.000512 : 0.000512 | True] -> 
[0.004096 : 0.004096 | True] -> 
[0.000128 : 0.000128 | True] -> 
[8e-06 : 8e-06 | True] -> 
[6.4e-05 : 6.4e-05 | True] -> 
[0.131072 : 0.131072 | True] -> 


In [ ]:
0.0005 
0.0007
0.0016
0.0004
0.0028

In [10]:
for exp in ["norm","cnorm","cnormu","gcstdu","noreg","wd"]:
    for hyp in os.listdir(os.path.join("./exps_adam2",exp)):
        with open(os.path.join("./exps_adam2",exp,hyp,"settings.json"),"r") as f:
            js = json.load(f)
        js["save_args"]["save_states_every"] = 200000
        js["save_args"]["save_grad_every"] = -1
        js["num_steps"] = 200000
        js["model"]["features_div"] = 8

        os.makedirs(os.path.join("./exps_adam2",exp+"_div8",hyp))
        with open(os.path.join("./exps_adam2",exp+"_div8",hyp,"settings.json"),"w") as f:
            json.dump(js,f)

        print("python main.py {0}/ --reset".format(os.path.join("./exps_adam2",exp+"_div8",hyp)))

python main.py ./exps_adam2/norm_div8/0.8_1/ --reset
python main.py ./exps_adam2/norm_div8/2.0_1/ --reset
python main.py ./exps_adam2/norm_div8/3.6_1/ --reset
python main.py ./exps_adam2/norm_div8/0.4_1/ --reset
python main.py ./exps_adam2/norm_div8/2.2_1/ --reset
python main.py ./exps_adam2/norm_div8/1.8_1/ --reset
python main.py ./exps_adam2/norm_div8/3.4_1/ --reset
python main.py ./exps_adam2/norm_div8/1.2_1/ --reset
python main.py ./exps_adam2/norm_div8/0.2_1/ --reset
python main.py ./exps_adam2/norm_div8/3.0_1/ --reset
python main.py ./exps_adam2/norm_div8/2.4_1/ --reset
python main.py ./exps_adam2/norm_div8/1.0_1/ --reset
python main.py ./exps_adam2/norm_div8/2.6_1/ --reset
python main.py ./exps_adam2/norm_div8/2.8_1/ --reset
python main.py ./exps_adam2/norm_div8/0.6_1/ --reset
python main.py ./exps_adam2/norm_div8/1.6_1/ --reset
python main.py ./exps_adam2/norm_div8/3.2_1/ --reset
python main.py ./exps_adam2/norm_div8/1.4_1/ --reset
python main.py ./exps_adam2/cnorm_div8/0.8_1/ 

In [8]:
start_wd = 0.000001
for i in range(1,8):
    with open(os.path.join("./exps_sgdm/wd","0.001","settings.json"),"r") as f:
        js = json.load(f)
    js["optimizer"]["optimizer"] = "adam"
    js["save_args"]["save_states_every"] = 200000
    js["save_args"]["save_grad_every"] = -1
    js["num_steps"] = 200000
    wd = round(start_wd/(2**i),8)
    js["optimizer"]["lambda_wd"] = wd
    wd = str(wd)
    os.makedirs(os.path.join("./exps_adam2/wd",wd))
    with open(os.path.join("./exps_adam2/wd",wd,"settings.json"),"w") as f:
        json.dump(js,f)

    print("python main.py {0}/ --reset".format(os.path.join("./exps_adam2/wd",wd)))


python main.py ./exps_adam2/wd/5e-07/ --reset
python main.py ./exps_adam2/wd/2.5e-07/ --reset
python main.py ./exps_adam2/wd/1.2e-07/ --reset
python main.py ./exps_adam2/wd/6e-08/ --reset
python main.py ./exps_adam2/wd/3e-08/ --reset
python main.py ./exps_adam2/wd/2e-08/ --reset
python main.py ./exps_adam2/wd/1e-08/ --reset


In [10]:
for hyp in os.listdir("./exps_adam2/noreg"):
    with open(os.path.join("./exps_adam2/noreg",hyp,"settings.json"),"r") as f:
        js = json.load(f)
    
    lr = js["optimizer"]["lr"]
    save_path = js["save_path"]
    save_path = save_path.split("/")
    new_name = str(round(lr,5))
    save_path[-2] = new_name 
    save_path = "/".join(save_path)

    with open(os.path.join("./exps_adam2/noreg",hyp,"settings.json"),"w") as f:
        json.dump(js,f)
    
    shutil.move(os.path.join("./exps_adam2/noreg",hyp),os.path.join("./exps_adam2/noreg",new_name) )

In [9]:
path = "./exps_sgdm"
for exp in os.listdir(path):

    if exp.endswith("_div8"):

        for sub_exp in os.listdir(os.path.join(path,exp)):
            if os.path.exists(os.path.join(path,exp,sub_exp,"settings.json")):
                with open(os.path.join(path,exp,sub_exp,"settings.json"),"r") as f:
                    js = json.load(f)
                
                js["model"]["features_div"] = 4
                os.makedirs(os.path.join(path,exp.replace("_div8","_div4"),sub_exp))
                with open(os.path.join(path,exp.replace("_div8","_div4"),sub_exp,"settings.json"),"w") as f:
                    json.dump(js,f)

                print("python main.py {0}/ --reset".format(os.path.join(path,exp.replace("_div8","_div4"),sub_exp)))


python main.py ./exps_sgdm/gcstdu_div4/0.4_1/ --reset
python main.py ./exps_sgdm/gcstdu_div4/0.2_1/ --reset
python main.py ./exps_sgdm/gcstdu_div4/0.25_1/ --reset
python main.py ./exps_sgdm/gcstdu_div4/0.05_1/ --reset
python main.py ./exps_sgdm/gcstdu_div4/0.3_1/ --reset
python main.py ./exps_sgdm/gcstdu_div4/0.45_1/ --reset
python main.py ./exps_sgdm/gcstdu_div4/0.1_1/ --reset
python main.py ./exps_sgdm/gcstdu_div4/0.15_1/ --reset
python main.py ./exps_sgdm/gcstdu_div4/0.5_1/ --reset
python main.py ./exps_sgdm/gcstdu_div4/0.35_1/ --reset
python main.py ./exps_sgdm/wd_div4/0.014/ --reset
python main.py ./exps_sgdm/wd_div4/0.005/ --reset
python main.py ./exps_sgdm/wd_div4/0.011/ --reset
python main.py ./exps_sgdm/wd_div4/0.001/ --reset
python main.py ./exps_sgdm/wd_div4/0.006/ --reset
python main.py ./exps_sgdm/wd_div4/0.002/ --reset
python main.py ./exps_sgdm/wd_div4/0.009/ --reset
python main.py ./exps_sgdm/wd_div4/0.007/ --reset
python main.py ./exps_sgdm/wd_div4/0.004/ --reset
pytho

In [8]:
target_folder = "cnorm"
target_scheme = "center_norm"
for hyperparam in os.listdir(os.path.join("./exps_adam2","norm")):
    with open(os.path.join("./exps_adam2","norm",hyperparam,"settings.json"),"r") as f:
        js = json.load(f)
    
    js["norm"]["norm_fn"] = target_scheme
    os.makedirs(os.path.join("./exps_adam2",target_folder,hyperparam),exist_ok=True)
    with open(os.path.join("./exps_adam2",target_folder,hyperparam,"settings.json"),"w") as f:
        json.dump(js,f)

    print("python main.py {0}/ --reset".format(os.path.join("./exps_adam2",target_folder,hyperparam)))

python main.py ./exps_adam2/cnorm/0.8_1/ --reset
python main.py ./exps_adam2/cnorm/2.0_1/ --reset
python main.py ./exps_adam2/cnorm/3.6_1/ --reset
python main.py ./exps_adam2/cnorm/0.4_1/ --reset
python main.py ./exps_adam2/cnorm/2.2_1/ --reset
python main.py ./exps_adam2/cnorm/1.8_1/ --reset
python main.py ./exps_adam2/cnorm/3.4_1/ --reset
python main.py ./exps_adam2/cnorm/1.2_1/ --reset
python main.py ./exps_adam2/cnorm/0.2_1/ --reset
python main.py ./exps_adam2/cnorm/3.0_1/ --reset
python main.py ./exps_adam2/cnorm/2.4_1/ --reset
python main.py ./exps_adam2/cnorm/1.0_1/ --reset
python main.py ./exps_adam2/cnorm/2.6_1/ --reset
python main.py ./exps_adam2/cnorm/2.8_1/ --reset
python main.py ./exps_adam2/cnorm/0.6_1/ --reset
python main.py ./exps_adam2/cnorm/1.6_1/ --reset
python main.py ./exps_adam2/cnorm/3.2_1/ --reset
python main.py ./exps_adam2/cnorm/1.4_1/ --reset


In [3]:
for i in ["0.2","0.4","3.0","3.2","3.4","3.6"]:
    with open(os.path.join("./exps_adam2","norm","0.6_1","settings.json"),"r") as f:
        js = json.load(f)
        js["optimizer"]["optimizer"] = "adam"
        js["save_args"]["save_states_every"] = 50000
        js["save_args"]["save_grad_every"] = 50000
        js["num_steps"] = 200000
        js["norm"]["norm_multiply"] = float(i)
    
    os.makedirs(os.path.join("./exps_adam2","norm",i+"_1"),exist_ok=True)
    with open(os.path.join("./exps_adam2","norm",i+"_1","settings.json"),"w") as f:
        json.dump(js,f)

    print("python main.py {0}/ --reset".format(os.path.join("./exps_adam2","norm",i+"_1")))


python main.py ./exps_adam2/norm/0.2_1/ --reset
python main.py ./exps_adam2/norm/0.4_1/ --reset
python main.py ./exps_adam2/norm/3.0_1/ --reset
python main.py ./exps_adam2/norm/3.2_1/ --reset
python main.py ./exps_adam2/norm/3.4_1/ --reset
python main.py ./exps_adam2/norm/3.6_1/ --reset


In [4]:
for i in ["0.2","0.4","3.0","3.2","3.4","3.6"]:
    with open(os.path.join("./exps_adam2","noreg","0.001","settings.json"),"r") as f:
        js = json.load(f)
        js["optimizer"]["optimizer"] = "adam"
        js["save_args"]["save_states_every"] = 50000
        js["save_args"]["save_grads_every"] = 50000
        js["num_steps"] = 200000
        js["optimizer"]["lr"] = 0.001/(float(i)**2)

        lr_str = str(round(0.001/(float(i)**2),5))
    os.makedirs(os.path.join("./exps_adam2","noreg",lr_str),exist_ok=True)
    with open(os.path.join("./exps_adam2","noreg",lr_str,"settings.json"),"w") as f:
        json.dump(js,f)

    print("python main.py {0}/ --reset".format(os.path.join("./exps_adam2","noreg",lr_str)))


python main.py ./exps_adam2/noreg/0.025/ --reset
python main.py ./exps_adam2/noreg/0.00625/ --reset
python main.py ./exps_adam2/noreg/0.00011/ --reset
python main.py ./exps_adam2/noreg/0.0001/ --reset
python main.py ./exps_adam2/noreg/9e-05/ --reset
python main.py ./exps_adam2/noreg/8e-05/ --reset


In [5]:
for folder in os.listdir("./exps_adam2/norm"):
    print("python main.py {0}/ ".format(os.path.join("./exps_adam2/norm", folder)))


python main.py ./exps_adam2/norm/0.4_1/ 
python main.py ./exps_adam2/norm/0.2_1/ 
python main.py ./exps_adam2/norm/0.25_1/ 
python main.py ./exps_adam2/norm/0.05_1/ 
python main.py ./exps_adam2/norm/0.3_1/ 
python main.py ./exps_adam2/norm/0.45_1/ 
python main.py ./exps_adam2/norm/0.1_1/ 
python main.py ./exps_adam2/norm/0.15_100/ 
python main.py ./exps_adam2/norm/0.15_1/ 
python main.py ./exps_adam2/norm/0.15_10/ 
python main.py ./exps_adam2/norm/0.5_1/ 
python main.py ./exps_adam2/norm/0.35_1/ 


In [10]:
for folder in os.listdir("./exps_sgdm/norm_dssmall"):
    print("python main.py {0}/ --reset".format(os.path.join("./exps_sgdm/norm_dssmall", folder)))

for folder in os.listdir("./exps_sgdm/wd_dssmall"):
    print("python main.py {0}/ --reset".format(os.path.join("./exps_sgdm/wd_dssmall", folder)))

python main.py ./exps_sgdm/norm_dssmall/0.15000000000000002_1/ --reset
python main.py ./exps_sgdm/norm_dssmall/0.4_1/ --reset
python main.py ./exps_sgdm/norm_dssmall/0.35000000000000003_1/ --reset
python main.py ./exps_sgdm/norm_dssmall/0.2_1/ --reset
python main.py ./exps_sgdm/norm_dssmall/0.25_1/ --reset
python main.py ./exps_sgdm/norm_dssmall/0.05_1/ --reset
python main.py ./exps_sgdm/norm_dssmall/0.3_1/ --reset
python main.py ./exps_sgdm/norm_dssmall/0.45_1/ --reset
python main.py ./exps_sgdm/norm_dssmall/0.1_1/ --reset
python main.py ./exps_sgdm/norm_dssmall/0.5_1/ --reset
python main.py ./exps_sgdm/wd_dssmall/0.014/ --reset
python main.py ./exps_sgdm/wd_dssmall/0.005/ --reset
python main.py ./exps_sgdm/wd_dssmall/0.011/ --reset
python main.py ./exps_sgdm/wd_dssmall/0.001/ --reset
python main.py ./exps_sgdm/wd_dssmall/0.010000000000000002/ --reset
python main.py ./exps_sgdm/wd_dssmall/0.013000000000000001/ --reset
python main.py ./exps_sgdm/wd_dssmall/0.006/ --reset
python main.py

In [11]:
import tensorflow as tf
import tensorflow_datasets as tfds

2025-01-14 16:44:45.429025: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-14 16:44:45.436666: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-14 16:44:45.445950: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-14 16:44:45.448988: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-14 16:44:45.456390: I tensorflow/core/platform/cpu_feature_guar

In [27]:
builder = tfds.builder("cifar10",data_dir="./datasets")
builder.download_and_prepare()
ds_train,ds_test = builder.as_dataset(split=["train", "test"])

solve_dict = lambda elem : (elem["image"],elem["label"])
ds_train,ds_test = ds_train.map(solve_dict),ds_test.map(solve_dict)

dtype = tf.dtypes.float32
cast = lambda img,lbl : (tf.cast(img,dtype),lbl)
ds_train,ds_test = ds_train.map(cast),ds_test.map(cast)

mean = tf.convert_to_tensor([0.32768, 0.32768, 0.32768],dtype=dtype)[None,None,:]
std = tf.convert_to_tensor([0.27755222, 0.26925606, 0.2683012 ],dtype=dtype)[None,None,:]

normalize = lambda img,lbl : ((img/255-mean)/std,lbl)

ds_train,ds_test = ds_train.map(normalize),ds_test.map(normalize)

ds_train = ds_train.shuffle(buffer_size=50000).take(1)

In [28]:
print(list(ds_train)[0][1])

tf.Tensor(1, shape=(), dtype=int64)
